In [ ]:
# This notebook contains preparation for Fault tests
# It creates and saves a fake cube with a fault
import os
import sys
import shutil
import warnings
import numpy as np

warnings.filterwarnings('ignore')

sys.path.insert(0, '../../../seismiqb')
sys.path.insert(0, '..')

from seismiqb import Fault, Field, array_to_segy
from utils import generate_synthetic

In [ ]:
""" You can manage notebook execution kwargs which relates to cube and fault for the test:

SYNTHETIC_MODE : bool
    Whether to create a synthetic data (cube and fault) or use existed, provided by paths.
CUBE_PATH : str or None
    Path to an existed seismic cube.
    Notice that it is only used with SYNTHETIC_MODE = False.
FAULT_PATH : str or None
    Path to an existed seismic fault.
    Notice that it is only used with SYNTHETIC_MODE = False.
CUBE_SHAPE : sequence of three integers
    Shape of a synthetic cube.
"""
# Synthetic creation parameters
SYNTHETIC_MODE = True
CUBE_PATH = None
FAULT_PATH = None
CUBE_SHAPE = (500, 500, 200)

# Visualization parameters
SCALE = 1
SHOW_FIGURES = True

# Prepare a workspace

Create directories for files and create a fake cube with faults and save cube and one fault.

**Storage structure:**
___



**fault_test_files** (fault tests directory with temporary files)

&emsp;├── **test_cube.sgy**

&emsp;├── **test_cube.meta**

&emsp;└── **test_fault**


In [ ]:
OUTPUT_DIR = './fault_test_files'

# (Re)create the test directory
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)

# Create a fake cube and a fake fault

In [ ]:
%%time
if SYNTHETIC_MODE:
    CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_cube.sgy')
    FAULT_PATH = os.path.join(OUTPUT_DIR, 'test_fault')

    # Create a cube and a fault
    synt3d, _, fault_sticks = generate_synthetic(shape=CUBE_SHAPE)

    # Dump data
    array_to_segy(synt3d, CUBE_PATH, zip_segy=False, delay=300)

    # Check cube
    field = Field(CUBE_PATH)

    geometry_loading_error = "Synthetic geometry creation error: saved and loaded data are different"
    assert np.allclose(field.geometry[:, :, :], synt3d), geometry_loading_error

    # Choose one fault and save it
    fault = Fault({'sticks': fault_sticks}, field=field, name='test')
    fault.dump_fault_sticks(FAULT_PATH)

    # Check fault
    opened_fault = Fault(FAULT_PATH, field=field)

    fault_loading_error = "Synthetic fault creation error: saved and loaded data are different"
    assert np.array_equal(
        np.concatenate(fault.sticks),
        np.concatenate(opened_fault.sticks)
    ), fault_loading_error

    opened_fault.show(axis=2, centering=True)
    
    opened_fault_without_transform = Fault(FAULT_PATH, field=field, transform=False, verify=False)

    assert ((opened_fault_without_transform.nodes - opened_fault.nodes)[:, :2] == [1, 1]).all(), "Wrong ilines/xlines offset"